In [3]:
# ==============================================================================
#  CONFIGURAÇÃO DO AMBIENTE E DO CAMINHO (EXECUTAR ESTA CÉLULA PRIMEIRO)
# ==============================================================================
import sys
import os

# Adiciona o diretório raiz do projeto ao Python Path
# Isso permite que o notebook encontre o módulo 'src'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Raiz do projeto adicionada ao sys.path: {project_root}")

Raiz do projeto adicionada ao sys.path: /home/douglas-souza/PycharmProjects/Smart Decisions in the Field: An AI and Data Science Guide


In [4]:
# --- 1. Importando nossas próprias ferramentas ---
# Agora, com o sys.path corrigido, estas importações funcionarão!
from src.data.preprocess import clean_abate_data, load_abate_data
from src.features.build_features import create_time_series_for_modeling
from src.models.train_model import split_time_series, train_sarima_model, forecast_model
from src.visualization.plot_results import plot_forecast_vs_actual

print("Módulos do projeto importados com sucesso!")

# (o resto do código do notebook continua aqui...)

Módulos do projeto importados com sucesso!


In [8]:
# notebooks/04_Modeling_Workflow.ipynb

# --- 0. CONFIGURAÇÃO DO AMBIENTE E DO CAMINHO ---
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print(f"Raiz do projeto adicionada ao sys.path: {project_root}")


# --- 1. IMPORTANDO NOSSAS PRÓPRIAS FERRAMENTAS ---
# Adicionamos as importações do nosso novo módulo de avaliação
from src.data.preprocess import clean_abate_data, load_abate_data
from src.features.build_features import create_time_series_for_modeling
from src.models.train_model import split_time_series, train_sarima_model, forecast_model
from src.visualization.plot_results import plot_forecast_vs_actual
from src.models.evaluate_model import calculate_forecasting_metrics, print_evaluation_report

print("Módulos do projeto importados com sucesso!")


# --- 2. DEFINIÇÃO DE CONSTANTES E CAMINHOS ---
RAW_FILE_PATH = os.path.join('..', 'data', 'raw', 'sigsifrelatorioabatesporanouf.csv')
CLEAN_FILE_PATH = os.path.join('..', 'data', 'processed', 'abate_animal_limpo.csv')
TARGET_ESPECIE = 'AVES'
TEST_MONTHS = 12


# --- 3. EXECUTANDO O PIPELINE COMPLETO ---
print("\n--- EXECUTANDO O PIPELINE DE MODELAGEM E AVALIAÇÃO DE PONTA A PONTA ---")

# Etapa de Pré-processamento
if not os.path.exists(CLEAN_FILE_PATH):
    print("Arquivo limpo não encontrado. Executando o pré-processamento...")
    raw_df = load_abate_data(RAW_FILE_PATH)
    if raw_df is not None:
        clean_df = clean_abate_data(raw_df)
        clean_df.to_csv(CLEAN_FILE_PATH, index=False)
        print(f"Arquivo limpo salvo em: {CLEAN_FILE_PATH}")
else:
    print(f"Usando arquivo limpo existente de: {CLEAN_FILE_PATH}")

# Etapa de Construção de Features
ts_target = create_time_series_for_modeling(CLEAN_FILE_PATH, TARGET_ESPECIE)

if ts_target is not None:
    # Etapa de Divisão dos Dados
    train, test = split_time_series(ts_target, test_size=TEST_MONTHS)

    # Etapa de Treinamento
    modelo = train_sarima_model(train)

    # Etapa de Previsão
    previsao = forecast_model(modelo, steps=len(test))

    # --- NOVA ETAPA DE AVALIAÇÃO QUANTITATIVA ---
    # Calcula as métricas comparando os valores de teste reais com os previstos
    metricas = calculate_forecasting_metrics(test, previsao)

    # Imprime o relatório de avaliação formatado
    print_evaluation_report(metricas)
    # ----------------------------------------------

    # Etapa de Visualização
    plot_forecast_vs_actual(train, test, previsao, model_name=f"SARIMA para {TARGET_ESPECIE}")

print("\n--- PIPELINE CONCLUÍDO ---")

Raiz do projeto adicionada ao sys.path: /home/douglas-souza/PycharmProjects/Smart Decisions in the Field: An AI and Data Science Guide
Módulos do projeto importados com sucesso!

--- EXECUTANDO O PIPELINE DE MODELAGEM E AVALIAÇÃO DE PONTA A PONTA ---
Usando arquivo limpo existente de: ../data/processed/abate_animal_limpo.csv
--- Construindo Features para a Espécie: AVES ---
Dados limpos carregados.
Série temporal para modelagem criada com sucesso.
Período: de 2001-01 a 2024-12
Número de pontos de dados: 288
Dividindo a série temporal: 276 para treino, 12 para teste.
Treinando o modelo SARIMA... Isso pode levar alguns minutos.
Treinamento do modelo SARIMA concluído.
                                     SARIMAX Results                                      
Dep. Variable:                         abate_aves   No. Observations:                  276
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 12)   Log Likelihood               -4555.919
Date:                            Sat, 07 Jun 2025   A

Gráfico interativo salvo em: /home/douglas-souza/PycharmProjects/Smart Decisions in the Field: An AI and Data Science Guide/src/visualization/../../reports/figures/forecast_sarima_para_aves.html

--- PIPELINE CONCLUÍDO ---
